In [30]:
import os
import re
import pandas as pd

# Limpeza dos discursos

## 1 Criando uma lista com todos os discursos

In [31]:
discursos = []
for j in os.listdir('discursos'):
    file = open(f'discursos/{j}', 'r', encoding="utf-8")
    #file = open(f'discursos/{j}', 'r', encoding="iso-8859-1") 
    text = file.read()
    discursos.append(text)
    file.close  


In [32]:
discursos = pd.Series(discursos)

In [33]:
discursos[0]

'[A SRA. CHRISTIANE DE SOUZA YARED ]+[(Bloco/PP - PR. Pela ordem. Sem revisão da oradora.) - Obrigada, Sr. Presidente., , Sr. Presidente, eu quero aqui pedir a todos os colegas, os nossos amigos Deputados e Senadores também, que nos permitam fazer com que vocês entendam qual é a nossa preocupação com nós não derrubarmos o Veto 12. É muito sério o que se passa neste país. Nós temos um número infinito de mortes e de tragédias de trânsito., , Esse veto vai de encontro a todos esses irresponsáveis que dirigem a mais de 200km/h nas estradas do país, às vezes alcançando 300km/h, filmando essas infrações, esses crimes de trânsito e causando a preocupação maior, porque nós infelizmente temos a cultura da casa arrombada - só se corre atrás depois que a casa foi arrombada. O que nós queremos é evitar essas tragédias, essas mortes. Essas famílias ficam completamente desoladas e perdem seus filhos por irresponsáveis., , É necessária a derrubada do Veto 12. Eu venho pedir aqui, conclamando todos vo

In [34]:
# codigo de verificação de ssl 
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [35]:
#Criando lista com todos os conectivos da lingua portuguesa
conectivos = pd.read_html('https://duvidas.dicio.com.br/conectivos/')

a = conectivos[0]['Exemplos:']
conectivos_lista = []
for j in a:
    conectivos_lista.append(j.upper())
conectivos_lista = " ".join(conectivos_lista)

conectivos_lista = conectivos_lista.replace('; ', ' ').replace('… ', ' ').split(' ')
conectivos_lista = list(dict.fromkeys(conectivos_lista))
outros_conectivos = ['OS', 'E', 'UM', 'UMA']
for i in outros_conectivos:
    conectivos_lista.append(i)
conectivos_lista = sorted(conectivos_lista)


## 2 Verificando se houve revisão nos discursos

In [36]:
revisado = []
for j in range(len(discursos)):
    if(re.search(r'Sem revisão', discursos[j])):
        revisado.append(False)
    else:
        revisado.append(True)
revisao = pd.Series(revisado)

In [37]:
df_discursos = {"discursos" : discursos,
                "revisao" : revisao}
df_discursos = pd.concat(df_discursos, axis=1)

## 3 Limpando os discursos

In [54]:
from urllib.request import AbstractBasicAuthHandler


df_discursos_limpos = []
for i in range(len(df_discursos['discursos'])):
    discurso = df_discursos['discursos'][i]

    # removendo a introdução
    discurso = discurso.replace((re.findall(r'(?<=^).*(?<=\))', discurso))[0], '')

    #Retirando os sons ambientes
    sons_ambiente = ['<tr><td align="center">(<i>Soa a campainha.</i>)</td></tr></table>', '<i>(Palmas.)</i>']
    for j in sons_ambiente:
        discurso = discurso.replace(j, '') 

    #Retirando as tags
    tags = ['<i>', '</i>', f'\n']
    for j in tags:
        discurso = discurso.replace(j, '')

    #Retirando pontuacoes
    pontuacoes = ['.', ':', '...', '(', ')', '[', ']', '{', '}', '!', '?', ',', ';', '-', '_', "'", "\"", "nº", '<', '>', '\\', '|', '/', ', , ', ']\n']
    for j in pontuacoes:
        discurso = discurso.replace(j, '')

    #retirando numeros
    for j in range(10):
        discurso = discurso.replace(str(j), '') 

    discurso =  discurso.split(' ')

    #separando os termos
    termos = []
    for j in range(len(discurso)):
        if discurso[j] != '':
            termos.append(discurso[j].upper())

    termos = {'termo' : pd.Series(sorted(termos))} 
    termos = pd.concat(termos, axis=1)
 

    for j in conectivos_lista:
        termos = termos[termos.termo != j]
    termos = termos.reset_index().drop(columns = ['index'])

    df_discursos_limpos.append(termos)

['[O SR. MÁRCIO MACÊDO ]+[(Bloco/PT - SE. Pela Liderança. Sem revisão do orador.) - Sr. Presidente Marcelo Ramos, Sras. Deputadas e Srs. Deputados, Senadores e Senadoras, eu utilizo o tempo da Liderança do meu partido, o PT, cedido pela Líder Reginaldo Lopes, para falar que estou voltando à Câmara dos Deputados depois de sete anos, com o mesmo entusiasmo, com a mesma alegria, com a mesma determinação e com a mesma honra com que tomei posse em 2011. , , A votação que tive na eleição de 2018 me permitiu estar entre os oito mais votados nas oito vagas destinadas ao Estado de Sergipe. A regra da sobra eleitoral utilizada naquela eleição me tirou a vaga que o povo legitimamente me outorgou. Hoje, pelas razões conhecidas, aqui estou. Mesmo que tardiamente, faço justiça ao voto soberano do povo e à democracia representativa. , , Convivi nesta Casa por 4 anos respeitando democraticamente todas as matizes ideológicas. Aqui fiz muitos amigos, de diferentes partidos, que guardo e carrego comigo a

In [39]:
df_discursos_limpos

[          termo
 0     ALCANCEOS
 1       ALCANÇA
 2    ALCANÇANDO
 3        ALGUÉM
 4        AMIGOS
 ..          ...
 156       VOCÊS
 157       VOCÊS
 158     VOLANTE
 159         VÃO
 160       VÍDEO
 
 [161 rows x 1 columns],
              termo
 0          ASSUNTO
 1            CALAR
 2        CONGRESSO
 3            CUSTA
 4              DAS
 5           DIANTE
 6             ESTE
 7             ESTE
 8        EXPRESSÃO
 9       FINANCEIRO
 10           GERAR
 11           GRAVE
 12  IRRESPONSÁVEIS
 13       LIBERDADE
 14           LUCRO
 15           MUITO
 16        OBRIGADA
 17         PESSOAS
 18      PRESIDENTE
 19              SR
 20           TRATA
 21            USAR
 22            VETO
 23            VIDA,
        termo
 0  TDTRTABLE,
          termo
 0      ACHAMOS
 1          ANO
 2    APRECIADO
 3    APRECIÁLO
 4     APRESSAR
 ..         ...
 70      SEQUER
 71     SESSÕES
 72       TENHO
 73       TODOS
 74  TRAMITAÇÃO
 
 [75 rows x 1 columns],
          termo
 0   

In [40]:
conectivos_lista[1]

'ACHO'

In [50]:
df_discursos['discursos'][1]

'[A SRA. SORAYA SANTOS ]+[(PL - RJ. Pela ordem. Sem revisão da oradora.) - Presidente, eu queria primeiro me dirigir a V. Exa., queria um minuto literalmente da sua atenção. É questão regimental. V. Exa. é um regimentalista., , Ninguém destacou esse veto. Na verdade, eu imaginei até que, pela bandeira da Deputada Chris Yared, que nos representa a todos, que representa as mães e os pais que sofrem neste país por mortes de filhos, por violência de trânsito... E eu fiquei muito serena, achando que o PP tivesse feito destaque a esse Veto 12. E é por isso que eu quero primeiro me dirigir a V. Exa., porque, se houver alguma possibilidade regimental de V. Exa., por acordo de Líderes, retirar esse destaque da votação do presente momento, é o que eu gostaria de pedir a V. Exa., , Esse veto é gravíssimo, Sr. Presidente. Uma coisa é liberdade de expressão. Nós não estamos discutindo liberdade de expressão, porque nós somos a favor da liberdade de expressão; mas nós estamos falando de pessoas que 

In [42]:
termos_discurso = []
for i in range(len(df_discursos['discursos'])):
    termos_lista = list(df_discursos_limpos[i]['termo'])
    termos_discurso.append(termos_lista)

termos_discurso

[['ALCANCEOS',
  'ALCANÇA',
  'ALCANÇANDO',
  'ALGUÉM',
  'AMIGOS',
  'ANOS',
  'AQUI',
  'AQUI',
  'ARROMBADA',
  'ARROMBADA',
  'ASSISTA',
  'ATROPELANDO',
  'ATRÁS',
  'ATRÁS',
  'CAMINHÃO',
  'CASA',
  'CASA',
  'CAUSANDO',
  'CIRCULAÇÃO',
  'COLEGAS',
  'COLOCAREM',
  'COMPLETAMENTE',
  'COMPREENDEM',
  'CONCLAMANDO',
  'CONSCIÊNCIA',
  'CORRE',
  'CORRENDO',
  'CRIANÇA',
  'CRIMES',
  'CULTURA',
  'DEPUTADOS',
  'DERRUBADA',
  'DERRUBADA',
  'DERRUBARMOS',
  'DESOLADAS',
  'DIRIGEM',
  'ENCONTRO',
  'ENTENDAM',
  'ESSAS',
  'ESSAS',
  'ESSAS',
  'ESSAS',
  'ESSAS',
  'ESSE',
  'ESSES',
  'ESSES',
  'ESSES',
  'ESSES',
  'ESSES',
  'ESTA',
  'ESTAR',
  'ESTIVER',
  'ESTRADAS',
  'EU',
  'EU',
  'EU',
  'EVITAR',
  'FAMÍLIAS',
  'FAZER',
  'FAZER',
  'FICAM',
  'FILHOS',
  'FILMANDO',
  'FIQUE',
  'FOI',
  'FRENTE',
  'GENTE',
  'INDIGNADO',
  'INFELIZMENTE',
  'INFINITO',
  'INFRAÇÕES',
  'IRRESPONSÁVEIS',
  'IRRESPONSÁVEIS',
  'IRRESPONSÁVEIS',
  'JOVENS',
  'KMH',
  'KMH',
  'KM

In [49]:
df_discursos["Termo Discurso"] = termos_discurso
df_discursos["Termo Discurso"][1]


['ASSUNTO',
 'CALAR',
 'CONGRESSO',
 'CUSTA',
 'DAS',
 'DIANTE',
 'ESTE',
 'ESTE',
 'EXPRESSÃO',
 'FINANCEIRO',
 'GERAR',
 'GRAVE',
 'IRRESPONSÁVEIS',
 'LIBERDADE',
 'LUCRO',
 'MUITO',
 'OBRIGADA',
 'PESSOAS',
 'PRESIDENTE',
 'SR',
 'TRATA',
 'USAR',
 'VETO',
 'VIDA']

In [45]:
df_discursos_limpos[2]

,termo
0,TDTRTABLE


In [46]:
df_discursos['discursos'][0]

'[A SRA. CHRISTIANE DE SOUZA YARED ]+[(Bloco/PP - PR. Pela ordem. Sem revisão da oradora.) - Obrigada, Sr. Presidente., , Sr. Presidente, eu quero aqui pedir a todos os colegas, os nossos amigos Deputados e Senadores também, que nos permitam fazer com que vocês entendam qual é a nossa preocupação com nós não derrubarmos o Veto 12. É muito sério o que se passa neste país. Nós temos um número infinito de mortes e de tragédias de trânsito., , Esse veto vai de encontro a todos esses irresponsáveis que dirigem a mais de 200km/h nas estradas do país, às vezes alcançando 300km/h, filmando essas infrações, esses crimes de trânsito e causando a preocupação maior, porque nós infelizmente temos a cultura da casa arrombada - só se corre atrás depois que a casa foi arrombada. O que nós queremos é evitar essas tragédias, essas mortes. Essas famílias ficam completamente desoladas e perdem seus filhos por irresponsáveis., , É necessária a derrubada do Veto 12. Eu venho pedir aqui, conclamando todos vo

In [47]:
 #retirando identificacao 
id = df_discursos['discursos']
id = id.str.split('.\) - ', expand=True)

In [48]:
df_discursos['discursos novos'][1] #verificando separação

KeyError: 'discursos novos'

In [ ]:
#adicionei uma coluna nova sem querer
df_discursos.drop(columns=['discurso novo'])

,discursos,revisao,Termo Discurso,discursos novos
0,[A SRA. CHRISTIANE DE SOUZA YARED ]+[(Bloco/PP...,False,"[ALCANCEOS, ALCANÇA, ALCANÇANDO, ALGUÉM, AMIGO...","Obrigada, Sr. Presidente., , Sr. Presidente, e..."
1,[A SRA. SORAYA SANTOS ]+[(PL - RJ. Pela ordem....,False,"[ASSUNTO, CALAR, CONGRESSO, CUSTA, DAS, DIANTE...","Presidente, eu queria primeiro me dirigir a V...."
2,[O SR. MÁRCIO MACÊDO ]+[(Bloco/PT - SE. Pela L...,False,[TDTRTABLE],"Sr. Presidente Marcelo Ramos, Sras. Deputadas ..."
3,[O SR. TIAGO MITRAUD ]+[(NOVO - MG. Para discu...,False,"[ACHAMOS, ANO, APRECIADO, APRECIÁLO, APRESSAR,...","Perfeito. Obrigado, Presidente. , , Sei que o ..."
4,[O SR. BIRA DO PINDARÉ ]+[(Bloco/PSB - MA. Pel...,False,"[ABERTURA, ALCKMIN, ALIADOS, BRASIL, CAMINHAR,...","Sr. Presidente, Sras. e Srs. Parlamentares, eu..."
...,...,...,...,...
95,[O SR. AGUINALDO RIBEIRO ]+[(Bloco/PP - PB. Pe...,False,"[ACREDITA, ALTERNATIVAS, AQUI, AÍ, CAMINHO, CA...","Sr. Presidente, Deputado Marcelo, eu queria pr..."
96,[O SR. RICARDO BARROS ]+[(Bloco/PP - PR. Pela ...,False,"[ADIANTADOS, AGRONEGÓCIO, ALAVANCA, APENAS, AQ...","Obrigado, Presidente., , Peço também o esforço..."
97,[O SR. DANIEL SILVEIRA ]+[(UNIÃO - RJ. Pela or...,False,"[ACEITAS, ACEITÁVEL, ACONTECENDO, ALEXANDRE, A...","Presidente, obrigado. , , Eu quero só notifica..."
98,[O SR. IVAN VALENTE ]+[(PSOL - SP. Pela Lidera...,False,"[AGRADEÇO, CONCLUIR]","Obrigado., , Sr. Presidente, Sras. e Srs. Parl..."
